### 获取全部评论

In [2]:
# coding: utf-8
import re 
import pandas as pd
import requests
import random
import json
from urllib import parse
from urllib import request
import time
from datetime import datetime
from datetime import timedelta
header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Connection':'keep-alive'
}
cookies = '__mta=147569288.1539613293789.1539613310187.1539616722700.3; v=3; \
        _lxsdk_cuid=166781a789ec8-0139751cb2ee6e-8383268-144000-166781a789fc8; \
        __mta=147569288.1539613293789.1539613293789.1539613310187.2; uuid_n_v=v1; \
        iuuid=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; webp=true; ci=96%2C%E6%B5%8E%E5%8D%97; \
        _lxsdk=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; _lxsdk_s=166784ecee0-b1a-f0b-22%7C%7C2'
cookie = {}
for line in cookies.split(';'):
    name, value = cookies.strip().split('=', 1)
    cookie[name] = value
    
    


In [3]:
def get_id_by_name(name):
    url = 'http://maoyan.com/query?'
    dict1 ={'kw': name}
    url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
    html = requests.get(url=url+url_data, cookies=cookie,headers=header).content.decode('utf-8')
    movie_item = re.findall(r'data-val=\"{movieid:(.*?)}">', html)
    if len(movie_item)==0:
        return -1
    else:
        return movie_item[0]

In [4]:
def get_release_time_by_name(name):
    id = get_id_by_name(name)
    if id == -1:
        return None
    else:
        url = 'http://maoyan.com/films/'+get_id_by_name(name)
#         print(url)
        html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
        s = r'<li class=\"ellipsis\">(.*?)映<\/li>'
        release_time = re.findall(s, html)
        return release_time[0][0:10]#只截取时间 如 2018-09-30 

In [5]:
# def get_comments_all(name):
#     comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
#     start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # 获取当前时间，从当前时间向前获取
#     release_time = str(get_release_time_by_name(name))
#     end_time =  release_time+' 00:00:00'
#     count=0
#     comments.to_csv('all_'+name+'.csv',index=False,encoding="utf_8_sig")
# #     for i in range(0,2):
#     while start_time > end_time:  
#         try:
#             time.sleep(0.2)
            
#             url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
#             print(str(count)+"  "+url)
#             count = count+1
#             html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
#             data = json.loads(html)['cmts']
           
#             start_time = data[len(data)-1]['startTime']  # 获得末尾评论的时间
#             start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
#             start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
 
#             for item in data:
#                 try:
#                     comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
#                                                     'score':str(item['score']),'comment':item['content'],
#                                                 'nick':item['nick'],'gender':item['gender']},
#                                                    ignore_index=True)
#                 except:
#                     comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
#                                                     'score':str(item['score']),'comment':item['content'],
#                                                 'nick':item['nick'],'gender':0},
#                                                    ignore_index=True)
#             comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
#             comments.drop(comments.index,inplace=True)#清空comments   
                    
#         except Exception as e:
#             time.sleep(0.5)
#             print(e)
#             continue


In [6]:
def get_comments_all(name):
    comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
    start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # 获取当前时间，从当前时间向前获取
    release_time = str(get_release_time_by_name(name))
    end_time =  release_time+' 00:00:00'
    comments.to_csv('all_'+name+'.csv',index=False,encoding="utf_8_sig")
    count=0
    pages =15
    
    while pages ==15:  
        try:
#             time.sleep(0.2)
            
            url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
            print(str(count)+"  "+url)
            count = count+1
            html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
            data = json.loads(html)['cmts']
            pages=len(data)

            for item in data:
                if item.get('cityName')==None or item.get('time')==None or item.get('score')==None or item.get('content')==None or item.get('nick')==None:
                    print("jump")
                else:  
                    try:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':item['gender']},
                                                       ignore_index=True)
                    except:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':0},
                                                       ignore_index=True)
            comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
            
            start_time = data[14]['startTime']  # 获得末尾评论的时间
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
            start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
            
            comments.drop(comments.index,inplace=True)#清空comments   

        except Exception as e:
            time.sleep(0.5)
            print(e)
            continue


In [7]:
def get_comments_continue(name,start_time):
    comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
    release_time = str(get_release_time_by_name(name))
    end_time =  release_time+' 00:00:00'
    count=0
    pages =15
    
    url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time
    html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
    data = json.loads(html)['cmts']
    pages=len(data)
    start_time = data[14]['startTime']  # 获得末尾评论的时间
    start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
    start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
    
    while pages ==15:  
        try:
            time.sleep(0.2)
            
            url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
            print(str(count)+"  "+url)
            count = count+1
            html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
            data = json.loads(html)['cmts']
            pages=len(data)

            for item in data:
                if item.get('cityName')==None or item.get('time')==None or item.get('score')==None or item.get('content')==None or item.get('nick')==None:
                    print("jump")
                else:  
                    try:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':item['gender']},
                                                       ignore_index=True)
                    except:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':0},
                                                       ignore_index=True)
            comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
            
            start_time = data[14]['startTime']  # 获得末尾评论的时间
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
            start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
            comments.drop(comments.index,inplace=True)#清空comments   

        except Exception as e:
            time.sleep(0.5)
            print(e)
            continue

In [ ]:
# get_comments_all("无双")

In [1]:
# get_comments_all("李茶的姑妈") #3718

In [2]:
# get_comments_all("影") #3729

In [63]:
# get_comments_all("找到你") #10月5号上映

In [ ]:
# get_comments_all("胖子行动队") #9月30上映

In [16]:
# get_comments_all("嘻哈英熊")

In [18]:
# get_comments_all("新灰姑娘")

In [20]:
# get_comments_all("阿凡提之奇缘历险")

In [12]:
# get_comments_all("夏洛特烦恼") #70w

0  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-10-20%2000:35:03
1  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-10-02%2016:06:02
jump
2  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-09-07%2020:09:31
3  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-08-27%2016:43:10
4  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-08-12%2012:14:43
5  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-08-05%2020:07:07
6  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-07-29%2018:34:33
7  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-07-21%2002:18:53
8  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2018-07-03%2020:17:27
9  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=

KeyboardInterrupt: 

In [8]:
# get_comments_all("驴得水") #6w

0  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-10-19%2015:19:47
1  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-09-03%2017:15:57
2  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-07-29%2013:46:47
3  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-07-04%2017:24:58
4  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-05-27%2021:24:51
5  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-04-22%2017:01:32
6  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-04-04%2016:39:03
7  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-03-12%2011:35:38
8  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&offset=0&startTime=2018-02-22%2022:40:00
9  http://m.maoyan.com/mmdb/comments/movie/410612.json?_v_=yes&o

KeyboardInterrupt: 

In [ ]:
# get_comments_all("羞羞的铁拳")

In [8]:
# get_comments_all("西虹市首富") #25943

In [16]:
get_comments_continue("夏洛特烦恼",'2016-07-08%2007:29:04')

0  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2023:42:39
1  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2021:42:45
jump
jump
2  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2019:53:21
3  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2018:38:39
jump
4  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2016:36:43
jump
5  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2014:26:31
jump
6  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2012:56:49
jump
7  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2011:39:52
jump
8  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-07%2010:21:07
jump
9  http://m.maoyan.com/m

74  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-01%2013:51:31
75  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-01%2012:29:29
jump
76  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-01%2011:34:12
jump
jump
77  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-01%2010:27:41
jump
jump
78  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-07-01%2008:20:57
jump
jump
jump
jump
79  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-30%2022:57:56
jump
80  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-30%2020:35:34
jump
jump
81  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-30%2018:41:37
82  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-30%2017:1

jump
jump
jump
148  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-24%2011:05:27
jump
jump
149  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-24%2010:07:50
150  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-24%2000:41:13
151  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2022:04:02
152  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2020:22:29
153  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2018:57:08
jump
jump
jump
154  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2017:31:26
155  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2016:20:50
156  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-23%2015:06:39
jump
1

jump
222  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2019:51:12
223  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2018:57:09
jump
224  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2016:50:02
jump
225  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2015:53:39
226  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2013:38:01
227  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2012:11:23
jump
jump
jump
228  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2010:50:47
jump
jump
229  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-17%2007:39:12
jump
230  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-16%2023:44:50
j

296  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-11%2012:44:19
297  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-11%2011:52:55
jump
298  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-11%2010:58:44
jump
jump
299  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-11%2010:11:52
300  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-11%2008:07:41
jump
jump
301  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-10%2023:38:58
302  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-10%2022:33:04
303  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-10%2021:23:59
304  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-06-10%2020:25:06
jump
305  http://m.ma

...

jump
jump
jump
jump
1620  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2015:54:32
jump
1621  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2015:36:36
jump
1622  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2015:09:54
jump
1623  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2014:47:02
jump
1624  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2013:52:49
jump
jump
jump
1625  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2013:23:56
jump
jump
1626  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2012:47:05
jump
jump
1627  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-04-02%2012:29:46
jump
jump
jump
1628  http://m.maoyan.com/mmdb/comments/movie/246082.j

1691  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2019:54:20
jump
1692  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2019:13:41
jump
jump
1693  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2018:39:31
jump
jump
jump
1694  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2017:46:17
1695  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2017:17:01
jump
1696  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2016:33:17
1697  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2016:08:44
jump
jump
jump
1698  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-30%2015:00:35
jump
jump
1699  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime

jump
jump
1763  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2016:07:58
jump
jump
1764  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2015:40:35
jump
1765  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2014:50:10
jump
1766  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2014:34:46
jump
jump
jump
1767  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2014:00:02
1768  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2013:31:13
jump
jump
1769  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2012:45:08
jump
jump
1770  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-27%2012:26:40
jump
jump
jump
1771  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=ye

jump
jump
1835  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2011:21:53
jump
1836  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2011:00:56
jump
jump
1837  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2010:23:41
jump
jump
1838  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2009:56:10
jump
jump
1839  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2008:30:31
jump
jump
1840  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2007:36:02
jump
1841  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2002:21:55
jump
1842  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&offset=0&startTime=2016-03-25%2000:12:34
jump
jump
1843  http://m.maoyan.com/mmdb/comments/movie/246082.json?_v_=yes&off

KeyboardInterrupt: 

In [9]:
# get_comments_continue("羞羞的铁拳",'2017-09-30%2012:11:16')

0  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:09:22
1  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:07:30
2  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:05:44
3  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:03:59
4  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:02:06
5  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2012:00:42
6  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2011:59:45
7  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2011:58:10
8  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-30%2011:56:52
9  http://m.maoyan.com/mmdb/comments/movie/1198214.json

78  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-16%2020:32:28
79  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-16%2015:58:31
80  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-16%2000:15:36
81  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-15%2013:59:39
82  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-14%2017:27:20
83  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-13%2013:05:25
84  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-10%2023:53:49
85  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-07%2021:58:24
86  http://m.maoyan.com/mmdb/comments/movie/1198214.json?_v_=yes&offset=0&startTime=2017-09-05%2011:43:17
87  http://m.maoyan.com/mmdb/comments/movie/11

In [ ]:
# get_comments_continue("西虹市首富",'2018-07-28%2000:00:00')

In [ ]:
# get_comments_continue("影",'2018-09-30%2000:00:00')

In [ ]:
# get_comments_continue("无双",'2018-09-30%2000:00:00')

In [11]:
# get_comments_continue("李茶的姑妈",'2018-09-26%2010:03:48')

0  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-26%2008:14:43
1  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-26%2001:33:07
2  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-26%2000:15:55
3  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2023:34:07
4  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2023:15:32
5  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2022:57:03
6  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2022:43:00
7  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2022:26:44
8  http://m.maoyan.com/mmdb/comments/movie/1217402.json?_v_=yes&offset=0&startTime=2018-09-25%2022:16:56
9  http://m.maoyan.com/mmdb/comments/movie/1217402.json